In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import dlib
import pandas as pd
from collections import Counter
from PIL import Image, ImageColor
import colorsys
import os

# Helper function to display images in the notebook
def display_img(img, figsize=(10, 10)):
    plt.figure(figsize=figsize)
    if len(img.shape) == 3:  # Color image
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:  # Grayscale
        plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()

# Function to check if the image file exists
def check_image_exists(image_path):
    if not os.path.exists(image_path):
        print(f"Error: The file at {image_path} does not exist.")
        return False
    return True

# Function to detect faces in an image
def detect_face(image_path):
    # Check if the image exists
    if not check_image_exists(image_path):
        return None, None, None

    # Load the image
    img = cv2.imread(image_path)
    
    # Convert to RGB (dlib works better with RGB)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Load the face detector from dlib
    detector = dlib.get_frontal_face_detector()
    
    # Load the facial landmark predictor
    predictor_path = "/Users/harshitdixit/Downloads/shape_predictor_68_face_landmarks.dat"  # Download this from dlib website
    try:
        predictor = dlib.shape_predictor(predictor_path)
    except RuntimeError:
        print(f"Error: Could not find the predictor file at {predictor_path}")
        print("Please download it from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
        return None, None, None
    
    # Detect faces
    faces = detector(rgb_img)
    
    if len(faces) == 0:
        print("No face detected in the image.")
        return None, None, img
    
    # Get the largest face (assuming the main subject)
    largest_face = max(faces, key=lambda rect: rect.width() * rect.height())
    
    # Get facial landmarks
    landmarks = predictor(rgb_img, largest_face)
    
    # Create a mask for the face
    mask = np.zeros_like(rgb_img[:,:,0])
    
    # Fill in the face region
    face_points = []
    for i in range(0, 68):
        x, y = landmarks.part(i).x, landmarks.part(i).y
        face_points.append((x, y))
    
    face_points = np.array(face_points, dtype=np.int32)
    cv2.fillPoly(mask, [face_points], 255)
    
    # Create a masked image containing only face pixels
    masked_face = cv2.bitwise_and(rgb_img, rgb_img, mask=mask)
    
    # Extract face from the image
    x, y, w, h = largest_face.left(), largest_face.top(), largest_face.width(), largest_face.height()
    face_img = rgb_img[y:y+h, x:x+w]
    
    return face_img, masked_face, img

# Function to extract even-lit face areas (avoiding shadows and highlights)
def extract_even_lit_areas(face_img, masked_face):
    if face_img is None:
        return None
    
    # Convert to LAB color space
    lab_img = cv2.cvtColor(masked_face, cv2.COLOR_RGB2LAB)
    
    # Extract L channel (luminance)
    l_channel = lab_img[:,:,0]
    
    # Keep only pixels with moderate luminance (avoiding shadows and highlights)
    min_lum = 50  # Avoid deep shadows
    max_lum = 200  # Avoid highlights
    
    # Create mask for even-lit areas
    even_mask = np.logical_and(l_channel >= min_lum, l_channel <= max_lum)
    even_mask = np.logical_and(even_mask, masked_face[:,:,0] > 0)  # Combine with face mask
    
    # Apply mask to get even-lit face areas
    even_lit_face = np.zeros_like(masked_face)
    even_lit_face[even_mask] = masked_face[even_mask]
    
    return even_lit_face

# Function to determine skin undertone
def analyze_skin_undertone(even_lit_face):
    if even_lit_face is None:
        return "Unknown"
    
    # Extract only the non-black pixels
    pixels = even_lit_face[np.any(even_lit_face != [0, 0, 0], axis=-1)]
    
    if len(pixels) == 0:
        return "Unknown"
    
    # Calculate average RGB
    avg_color = np.mean(pixels, axis=0)
    r, g, b = avg_color
    
    # Analyze red vs blue channels to determine warm vs cool
    if r > b:
        return "Warm"
    else:
        return "Cool"

# Function to analyze skin contrast level
def analyze_contrast(even_lit_face):
    if even_lit_face is None:
        return "Unknown"
    
    # Extract only the non-black pixels
    pixels = even_lit_face[np.any(even_lit_face != [0, 0, 0], axis=-1)]
    
    if len(pixels) == 0:
        return "Unknown"
    
    # Convert to grayscale
    gray_pixels = np.dot(pixels, [0.299, 0.587, 0.114])
    
    # Calculate standard deviation of luminance
    std_dev = np.std(gray_pixels)
    
    # Classify contrast
    if std_dev < 15:
        return "Low"
    elif std_dev < 30:
        return "Medium"
    else:
        return "High"

# Function to extract dominant colors
def extract_skin_colors(even_lit_face, n_colors=5):
    if even_lit_face is None:
        return []
    
    # Extract only the non-black pixels
    mask = np.any(even_lit_face != [0, 0, 0], axis=-1)
    pixels = even_lit_face[mask].reshape(-1, 3)
    
    if len(pixels) == 0:
        return []
    
    # Use KMeans to find dominant colors
    kmeans = KMeans(n_clusters=n_colors)
    kmeans.fit(pixels)
    
    # Get the colors
    colors = kmeans.cluster_centers_
    
    # Convert to integer RGB values
    colors = colors.astype(int)
    
    return colors

# Function to calculate color similarity (Euclidean distance in RGB space)
def color_distance(color1, color2):
    return np.sqrt(np.sum((color1 - color2) ** 2))

# Function to find complementary colors
def get_complementary_colors(skin_colors, undertone, contrast):
    complementary_colors = []
    
    # Color palettes based on undertone and contrast
    warm_palette = {
        "Low": [
            (255, 166, 158),  # Peach
            (255, 214, 153),  # Light Gold
            (204, 153, 102),  # Camel
            (153, 204, 153),  # Sage Green
            (204, 204, 153)   # Olive
        ],
        "Medium": [
            (204, 102, 0),    # Burnt Orange
            (204, 153, 0),    # Gold
            (153, 102, 51),   # Brown
            (102, 153, 102),  # Forest Green
            (153, 102, 102)   # Terracotta
        ],
        "High": [
            (204, 51, 0),     # Tomato Red
            (153, 102, 0),    # Dark Gold
            (102, 51, 0),     # Deep Brown
            (51, 102, 51),    # Deep Forest Green
            (102, 51, 51)     # Burgundy
        ]
    }
    
    cool_palette = {
        "Low": [
            (204, 204, 255),  # Lavender
            (153, 204, 204),  # Powder Blue
            (153, 153, 204),  # Periwinkle
            (204, 153, 204),  # Lilac
            (204, 204, 204)   # Silver
        ],
        "Medium": [
            (102, 102, 204),  # Blue-Purple
            (102, 153, 153),  # Teal
            (102, 102, 153),  # Purple-Gray
            (153, 102, 153),  # Mauve
            (153, 153, 153)   # Gray
        ],
        "High": [
            (51, 51, 153),    # Royal Blue
            (0, 102, 102),    # Dark Teal
            (51, 51, 102),    # Navy
            (102, 51, 102),   # Purple
            (51, 51, 51)      # Charcoal
        ]
    }
    
    # Get appropriate palette
    if undertone == "Warm":
        palette = warm_palette.get(contrast, warm_palette["Medium"])
    else:
        palette = cool_palette.get(contrast, cool_palette["Medium"])
    
    for color in palette:
        complementary_colors.append(color)
    
    return complementary_colors

# Function to convert RGB to hex
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

# Function to display color recommendations
def display_recommendations(complementary_colors, undertone, contrast):
    plt.figure(figsize=(15, 5))
    for i, color in enumerate(complementary_colors):
        plt.subplot(1, 5, i+1)
        plt.fill([0, 1, 1, 0], [0, 0, 1, 1], color=np.array(color)/255)
        hex_color = rgb_to_hex(color)
        plt.title(f"Color {i+1}\n{hex_color}")
        plt.axis('off')
    plt.suptitle(f"Recommended Colors for {undertone} Undertone, {contrast} Contrast", fontsize=16)
    plt.tight_layout()
    plt.show()

# Function to explain color recommendations
def explain_recommendations(complementary_colors, undertone, contrast):
    explanations = []
    
    color_descriptions = {
        # Warm colors
        (255, 166, 158): "Soft Peach - enhances natural warmth in skin with low contrast",
        (255, 214, 153): "Light Gold - brightens warm complexion without overwhelming",
        (204, 153, 102): "Camel - neutral that harmonizes with warm undertones",
        (153, 204, 153): "Sage Green - complements warm skin with a natural balance",
        (204, 204, 153): "Olive - earthy tone that enhances warm complexions",
        
        (204, 102, 0): "Burnt Orange - brings out the richness in medium contrast warm skin",
        (204, 153, 0): "Gold - highlights warmth in medium contrast complexions",
        (153, 102, 51): "Brown - grounds warm complexions with medium contrast",
        (102, 153, 102): "Forest Green - balances warmth with complementary depth",
        (153, 102, 102): "Terracotta - enhances natural flush in warm, medium contrast skin",
        
        (204, 51, 0): "Tomato Red - creates striking contrast that enhances high contrast warm skin",
        (153, 102, 0): "Dark Gold - adds richness to high contrast warm complexions",
        (102, 51, 0): "Deep Brown - anchors warm, high contrast coloring",
        (51, 102, 51): "Deep Forest Green - provides depth for high contrast warm skin",
        (102, 51, 51): "Burgundy - adds dramatic complement to warm, high contrast features",
        
        # Cool colors
        (204, 204, 255): "Lavender - softly enhances cool undertones with low contrast",
        (153, 204, 204): "Powder Blue - brightens cool, low contrast complexions",
        (153, 153, 204): "Periwinkle - adds gentle color that harmonizes with cool skin",
        (204, 153, 204): "Lilac - complements coolness while adding a touch of warmth",
        (204, 204, 204): "Silver - reflects light to enhance cool, low contrast features",
        
        (102, 102, 204): "Blue-Purple - brings depth to cool, medium contrast skin",
        (102, 153, 153): "Teal - balances cool undertones with medium contrast",
        (102, 102, 153): "Purple-Gray - sophisticated neutral for cool skin",
        (153, 102, 153): "Mauve - bridges cool and warm elements for medium contrast",
        (153, 153, 153): "Gray - classic neutral that enhances cool undertones",
        
        (51, 51, 153): "Royal Blue - creates striking complement to cool, high contrast features",
        (0, 102, 102): "Dark Teal - adds rich depth to cool, high contrast complexions",
        (51, 51, 102): "Navy - grounds cool coloring with sophisticated depth",
        (102, 51, 102): "Purple - dramatic color that enhances cool, high contrast skin",
        (51, 51, 51): "Charcoal - adds definition to cool, high contrast features"
    }
    
    # Get explanations for each color
    for i, color in enumerate(complementary_colors):
        # Convert tuple to rgb
        color_tuple = tuple(color)
        hex_color = rgb_to_hex(color)
        
        explanation = color_descriptions.get(color_tuple, f"Color {i+1} ({hex_color})")
        if color_tuple not in color_descriptions:
            # Add generic explanation based on undertone and contrast
            if undertone == "Warm":
                explanation += " - complements your warm undertones"
            else:
                explanation += " - enhances your cool undertones"
                
            if contrast == "Low":
                explanation += " with subtle harmony"
            elif contrast == "Medium":
                explanation += " with balanced contrast"
            else:
                explanation += " with striking definition"
        
        explanations.append(explanation)
    
    for i, explanation in enumerate(explanations, 1):
        print(f"{i}. {explanation}")

# Main function to process image and recommend colors
def analyze_face_and_recommend_colors(image_path):
    print("Loading and processing image...")
    face_img, masked_face, original_img = detect_face(image_path)
    
    if face_img is None:
        print("No face detected. Cannot perform color analysis.")
        return
    
    # Display original image
    print("Original Image:")
    display_img(original_img)
    
    # Display detected face
    print("Detected Face:")
    display_img(face_img)
    
    # Extract even-lit face areas
    print("Extracting evenly lit facial areas...")
    even_lit_face = extract_even_lit_areas(face_img, masked_face)
    
    if even_lit_face is None:
        print("Could not extract evenly lit facial areas. Cannot perform color analysis.")
        return
    
    # Display even-lit face areas
    print("Even-lit face areas (avoiding shadows and highlights):")
    display_img(even_lit_face)
    
    # Analyze skin undertone
    undertone = analyze_skin_undertone(even_lit_face)
    print(f"Skin Undertone: {undertone}")
    
    # Analyze contrast level
    contrast = analyze_contrast(even_lit_face)
    print(f"Contrast Level: {contrast}")
    
    # Extract skin colors
    skin_colors = extract_skin_colors(even_lit_face)
    print("Dominant skin colors:")
    for i, color in enumerate(skin_colors):
        print(f"Color {i+1}: RGB{tuple(color)} - HEX: {rgb_to_hex(color)}")
    
    # Get complementary colors
    complementary_colors = get_complementary_colors(skin_colors, undertone, contrast)
    
    # Display color recommendations
    print("\nRecommended Colors:")
    display_recommendations(complementary_colors, undertone, contrast)
    
    # Explain recommendations
    print("\nColor Recommendations Explained:")
    explain_recommendations(complementary_colors, undertone, contrast)

# Example usage
# To use this function, you need to download the facial landmark predictor from dlib
# http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
analyze_face_and_recommend_colors('/Users/harshitdixit/Downloads/test_user_image.png')